In [15]:
import os
os.chdir('/Users/ngdnhtien/Research/Codespace/PulsatingPulseShop/')

# Necessary dependencies
import qiskit
import qiskit_ibm_provider
import utility as ut
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import json

In [3]:
plt.rcParams['axes.linewidth'] = 2.0
%config InlineBackend.figure_formats = ['svg']

# Calibration for physical $SX^{12}$ gates

In this notebook I present the procedure for calibration of the elementary rotation generated by $\lambda_x^{(\text{s})}$.

### Connect to IBM Backend

In [24]:
provider = qiskit_ibm_provider.IBMProvider()
backend = provider.get_backend('ibm_brisbane')

### Selecting the qubit/cbit

In [25]:
qubit = 109
cbit = 0

num_qubits = int(qubit + 1)
num_cbits = 1

weight = 1

In [26]:
rough_01_freq = backend.defaults().qubit_freq_est[qubit]
qubit_anharmonicty = backend.properties().qubits[qubit][3].value * 1e9
rough_12_freq = rough_01_freq + qubit_anharmonicty

print(r"f01 = "+f'{round(rough_01_freq/1e9, 4)}'+' GHz')
print(r'f12 = '+f'{round(rough_12_freq/1e9, 4)}'+' GHz')

f01 = 4.985 GHz
f12 = 4.6779 GHz


### Reset circuits

In [27]:
reset_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
reset_circ.measure(qubit, cbit)

reset_circuits = [reset_circ, reset_circ, reset_circ]

## Step 1. Rabi oscillation

In [10]:
rabi_amplitude = np.linspace(-0.75, 0.75, 100)
rabi_gate = qiskit.circuit.Gate('rabi_gate', weight, [])
rabi_osc_circs = []

for amp in rabi_amplitude:

    with qiskit.pulse.build(backend=backend) as rabi_sched:
        drive_chan = qiskit.pulse.drive_channel(qubit)
        qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
        qiskit.pulse.play(qiskit.pulse.Drag(duration=64, amp=amp, sigma=16, beta=0), drive_chan)

    rabi_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
    rabi_circ.x(qubit) 
    rabi_circ.append(rabi_gate, [qubit])
    rabi_circ.measure(qubit, cbit)
    rabi_circ.add_calibration(rabi_gate, [qubit], rabi_sched)

    rabi_osc_circs.append(rabi_circ)

package = rabi_osc_circs + reset_circuits

#### Cautious

In [ ]:
rabi_job = backend.run(package, meas_level=1, meas_return='avg', shots=2048)
print(rabi_job.job_id())

In [12]:
rabi_job = provider.retrieve_job('crwf2vvdbt40008jxz9g')
print(rabi_job.status())

JobStatus.DONE


In [13]:
rabi_exp = ut.DataAnalysis(experiment=rabi_job, average=True, shots=2048, qubit=0)
rabi_exp.retrieve_data(average=True)
rabi_IQdata = np.array(rabi_exp.IQ_data) 

In [14]:
with open('./calibrator/ape_drag_sim/data/rabi_osc/'+f'{rabi_job.job_id()}'+'.pkl', 'wb') as f:
    pickle.dump(rabi_IQdata, f)

### Post-processing

In [18]:
with open("./calibrator/ape_drag_sim/sx12.json") as jf:
    sx12 = json.load(jf)

# Step 2. APE sequence + DRAG

In [28]:
reps = [5, 7, 9]
beta_values = np.linspace(-1.5, 0.5, 100)
ape_drag_circs = []

for rep in reps:
    for beta in beta_values:
        ape_drag_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
        ape_drag_circ.x(qubit)
        
        with qiskit.pulse.build(backend=backend) as hp12_sched:
            drive_chan = qiskit.pulse.drive_channel(qubit)
            qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
            qiskit.pulse.play(qiskit.pulse.Drag(duration=sx12['dur'], 
                                                amp=sx12['amp'], 
                                                sigma=sx12['sigma'], 
                                                beta=beta), drive_chan)
            
        hp12_gate = qiskit.circuit.Gate('hp12_gate', weight, []) 

        ape_drag_circ.append(hp12_gate, [qubit])
        
        with qiskit.pulse.build(backend=backend) as pseudo_I_sched:
            drive_chan = qiskit.pulse.drive_channel(qubit)
            qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
            qiskit.pulse.play(qiskit.pulse.Drag(duration=sx12['dur'], 
                                                amp=sx12['amp'], 
                                                sigma=sx12['sigma'], 
                                                beta=beta), drive_chan)
            qiskit.pulse.play(qiskit.pulse.Drag(duration=sx12['dur'], 
                                                amp=(-1) * sx12['amp'], 
                                                sigma=sx12['sigma'], 
                                                beta=beta), drive_chan)
            
        pseudo_I_gate = qiskit.circuit.Gate('pseudo_I_gate', weight, [])

        for j in range(rep):
            ape_drag_circ.append(pseudo_I_gate, [qubit])
        
        with qiskit.pulse.build(backend=backend) as minus_hp12_sched:
            drive_chan = qiskit.pulse.drive_channel(qubit)
            qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
            qiskit.pulse.play(qiskit.pulse.Drag(duration=sx12['dur'], 
                                                amp=(-1) * sx12['amp'], 
                                                sigma=sx12['sigma'], 
                                                beta=beta), drive_chan)
            
        minus_hp12_gate = qiskit.circuit.Gate('minus_hp12_gate', weight, [])
            
        ape_drag_circ.append(minus_hp12_gate, [qubit])
        ape_drag_circ.x(qubit)
        ape_drag_circ.measure(qubit, cbit)

        ape_drag_circ.add_calibration(hp12_gate, [qubit], hp12_sched)
        ape_drag_circ.add_calibration(pseudo_I_gate, [qubit], pseudo_I_sched)
        ape_drag_circ.add_calibration(minus_hp12_gate, [qubit], minus_hp12_sched)
        
        ape_drag_circs.append(ape_drag_circ)


In [29]:
with qiskit.pulse.build(backend=backend) as rough_sx12_sched:
    drive_chan = qiskit.pulse.drive_channel(qubit)
    qiskit.pulse.set_frequency(rough_12_freq, drive_chan)
    qiskit.pulse.play(qiskit.pulse.Drag(duration=sx12['dur'], 
                                        amp=sx12['amp'], 
                                        sigma=sx12['sigma'], 
                                        beta=0), drive_chan)
    
rough_sx12_gate = qiskit.circuit.Gate('rough_sx12_gate', weight, []) 

ground_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
ground_circ.measure(qubit, cbit)

first_excited_state_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
first_excited_state_circ.x(qubit)
first_excited_state_circ.measure(qubit, cbit)

second_excited_state_circ = qiskit.circuit.QuantumCircuit(num_qubits, num_cbits)
second_excited_state_circ.x(qubit)
second_excited_state_circ.append(rough_sx12_gate, [qubit])
second_excited_state_circ.append(rough_sx12_gate, [qubit])
second_excited_state_circ.measure(qubit, cbit)
second_excited_state_circ.add_calibration(rough_sx12_gate, [qubit], rough_sx12_sched)

discrim_circs = [ground_circ, first_excited_state_circ, second_excited_state_circ]

In [30]:
package = discrim_circs + ape_drag_circs + reset_circuits

In [31]:
ape_drag_job = backend.run(package, meas_level=1, meas_return='avg', shots=2048)
print(ape_drag_job.job_id())

crxhhx79c6ng008htthg


In [33]:
ape_drag_job = provider.retrieve_job('crxhhx79c6ng008htthg')
print(ape_drag_job.status())

JobStatus.QUEUED
